## Score matching 

we have data $p_{d}(x)$ and since this is diffcult to model directly, we want to fit a model $p_{m}(x, \theta)$. Here $\theta$ is the parameter of the model. As we train the model, we want to match the data distribution $p_{d}(x)$ as closely as possible.

we can write $p_{m}(x, \theta)$ as:

$$
p_m(x;\theta) = \frac{\tilde{p}(x;\theta)}{Z_\theta}, \quad Z_\theta = \int_\mathcal{X} \tilde{p}(x;\theta)dx.
$$


where $\tilde{p}$ is the unnormalized model distribution. Now since z is intractable we can remove this by derivating the above equation with respect to x 

$$
\nabla_{\mathbf{x}} \log p_m(x;\theta) = \nabla_{\mathbf{x}} \log \tilde{p}(x;\theta)  + \nabla_{\mathbf{x}} \log Z_\theta
$$

and since $z_\theta$ is independent of x, we can remove it.

$$
\nabla_{\mathbf{x}} \log p_m(x;\theta) = \nabla_{\mathbf{x}} \log \tilde{p}(x;\theta)
$$

so even though it is not the model distribution in its original form, score function gives the gradient of the model distribution. So here we will match the gradient of the model distribution with the gradient of the data distribution. Using fisher divergence we can write the difference between the two gradients as:

$$
\frac{1}{2}\mathbb{E}_{p_d}\left[\|\nabla_x \log p_d(x) - \nabla_x \log p_m(x;\theta)\|_2^2\right].
$$

and minimizing this is important to us. but computing this is difficult. The authors of the paper derived and wrote this in a simpler form as:

$$
L(\theta) \approx \frac{1}{n}\sum_{i=1}^n\left[\text{tr}\left(\nabla_x^2\log p_m(x_i;\theta)\right) + \frac{1}{2}\|\nabla_x\log p_m(x_i;\theta)\|_2^2\right]
$$

A detailed blog is [here]( https://andrewcharlesjones.github.io/journal/21-score-matching.html) on how we have arrived at this equation is here.

say we have a neural network model which takes an image as input and outputs a scalar score for each pixel.  then
- we can obtain the 2nd term by simply passing image into a neural network and square each value and sum them up. 
- 1st term is taking double graident, 1st gradient is obtained in the forward pass as above and the 2nd gradient is obtained by taking gradient of the gradient. This is called Hessian matrix for confusion but this is just jaccobian matrix when we consider this as derivative of output with respect to input. 







Now we will work on a toy example to understand this better.  Lets consider we have a 2D input and we want to fit a linear model to it with output having sigmoid.

## 1st layer 
$$
\begin{bmatrix} y_1 \\ y_2 \end{bmatrix} = \begin{bmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} + \begin{bmatrix} b_1 \\ b_2 \end{bmatrix}
$$

$$
 \begin{bmatrix} y_1 \\ y_2 \end{bmatrix} = \begin{bmatrix} w_{11} x_1 + w_{21} x_2 + b_1 \\ w_{12} x_1 + w_{22} x_2 + b_2 \end{bmatrix}
$$

## 2nd layer 
$$
\begin{bmatrix} y_1 \\ y_2 \end{bmatrix} = \begin{bmatrix} \sigma(y_1) \\ \sigma(y_2) \end{bmatrix}
$$


Now lets calculate the gradient for each element. In the above the first element is 

$$
y1 = \sigma(w_{11} x_1 + w_{21} x_2 + b_1)
$$

as we know the derivative of sigmoid is 

$$
\sigma'(x) = \sigma(x) (1 - \sigma(x))
$$

$$
\frac{\partial y_1}{\partial x_1} = \sigma (w_{11} x_1 + w_{21} x_2 + b_1) (1 - \sigma (w_{11} x_1 + w_{21} x_2 + b_1)) w_{11}
$$

$$
\frac{\partial y_2}{\partial x_2} = \sigma (w_{12} x_1 + w_{22} x_2 + b_2) (1 - \sigma (w_{12} x_1 + w_{22} x_2 + b_2)) w_{22}
$$


Now lets take some values and calculate this . we will take w_11 = 1, w_12 = 2, w_21 = 3, w_22 = 4, b_1 = 0, b_2 = 0, x_1 = 1, x_2 = 1

$$
\frac{\partial y_1}{\partial x_1} = \sigma (1 + 3 + 0) (1 - \sigma (1 + 3 + 0)) 1 = 0.017
$$

we will see in pytorch how to calculate this. 

In [8]:
import torch
import torch.nn as nn

# Simple score network
class ToyScoreNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 2)  # 2D input -> 2D output
        self.linear.weight.data = torch.tensor([[1, 3], [2, 4]]).float()
        self.linear.bias.data = torch.tensor([0, 0]).float()
        
    def forward(self, x):
        return torch.sigmoid(self.linear(x))  # Returns score: [∂/∂x₁ log p, ∂/∂x₂ log p]

# Create network and sample data
score_net = ToyScoreNetwork()
x = torch.tensor([[1.0, 1.0]], requires_grad=True)  # One 2D point

# Method 1: Computing full gradients
score = score_net(x)  # Shape: [1, 2]
print("Score output:", score)


Score output: tensor([[0.9820, 0.9975]], grad_fn=<SigmoidBackward0>)


In [11]:
torch.autograd.grad(
        score[:, 0],  # Take i-th component
        x,
        create_graph=True
    )[0][:, 0]

tensor([0.0177], grad_fn=<SelectBackward0>)

so if u see 0.017 term matches up with our original calculation. to calculate the overall loss we sum over the two terms and do the following

In [10]:
div = 0 
for i in range(2):  # For each dimension
    grad_i = torch.autograd.grad(
        score[:, i].sum(),  # Take i-th component
        x,
        create_graph=True
    )[0][:, i]  # Take i-th diagonal element
    div += grad_i
    print(f"Diagonal element {i}:", grad_i)
print("loss", div)

Diagonal element 0: tensor([0.0177], grad_fn=<SelectBackward0>)
Diagonal element 1: tensor([0.0099], grad_fn=<SelectBackward0>)
loss tensor([0.0275], grad_fn=<AddBackward0>)


so we got everything we are looking for, now lets train a model using MNIST dataset. 

> Note: In the above code, gradient is calculated with respect to each output score value. for a 32x32 image, we have 1024 output scores. so we will have 1024 gradients passing. This can be quite slow and time consuming. 

> So there is another paper called [sliced score matching] to solve this.

tensor([[ 0.0216, -0.7956]])